In [3]:
# add the parent directory to the path, so that we can import the package
import sys
sys.path.insert(0, '..')

In [261]:
import numpy
%load_ext autoreload
%autoreload 2
# will autoupdate any of the packages imported:
import pyclifford as pc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Helper Functions

In [63]:
def to_latex(obj):
    if isinstance(obj, pc.PauliMonomial):
        z = obj.c * (1j ** obj.p)
        if z == 0.:
            return "0"
        elif z.real == 0.:
            s = ""
        elif z == 1.:
            s = ""
        elif z == -1.:
            s = "-"
        else:
            s = str(z.real)
        if z.imag > 0:
            s += "+"
            if z.imag != 1.:
                s += str(z.imag)
            s += "i"
        elif z.imag < 0:
            if z.imag != -1.:
                s += str(z.imag)
            else:
                s += "-"
            s += "i"
        if len(s) > 0:
            s += " "
        for i in range(obj.N):
            if numpy.array_equal(obj.g[2*i:2*i+2], [1,0]):
                s += "X" + "_{" + str(i+1) + "}"
            elif numpy.array_equal(obj.g[2*i:2*i+2], [1,1]):
                s += "Y" + "_{" + str(i+1) + "}"
            elif numpy.array_equal(obj.g[2*i:2*i+2], [0,1]):
                s += "Z" + "_{" + str(i+1) + "}"
        if s[-1] == " ":
            s += "I"
        return s
    elif isinstance(obj, pc.PauliPolynomial):
        s = ""
        for k, term in enumerate(obj):
            s_term = to_latex(term)
            if k != 0:
                if s_term[0] == '-':
                    s_term = ' ' + s_term
                else:
                    s_term = ' +' + s_term
            s  = s + s_term
        return s
    elif isinstance(obj, pc.Pauli):
        return to_latex(obj.as_monomial())
    elif isinstance(obj, pc.PauliList):
        return [to_latex(op) for op in obj]
    else:
        return str(obj)


# Benchmark Problems for Agents

## I. Pauli Algebra

### I.1. Product of Pauli Strings

In [64]:
def prod_pauli(Nmin, samples, seed=42):
    rng = numpy.random.default_rng(seed)
    txt = ""
    for k in range(samples):
        N = round(Nmin + 1.3**k - 1)
        gs = rng.integers(0, 2, (2, 2*N))
        ps = rng.integers(0, 4, (2,))
        op0, op1 = pc.PauliList(gs, ps)
        op = op0 @ op1
        txt += "({}) ({}) = {}\n".format(to_latex(op0), to_latex(op1), to_latex(op))
    return txt
print(prod_pauli(1, 12))

(+i Z_{1}) (-i X_{1}) = +i Y_{1}
(- Z_{1}) (-i I) = +i Z_{1}
(- Y_{1}Y_{2}) (+i X_{1}X_{2}) = +i Z_{1}Z_{2}
(+i Z_{1}Y_{2}) (Z_{1}X_{2}) = Z_{2}
(Z_{1}X_{2}Y_{3}) (-i Z_{1}Z_{2}X_{3}) = +i Y_{2}Z_{3}
(+i Z_{1}Y_{2}X_{3}) (-i X_{2}Y_{3}Y_{4}) = Z_{1}Z_{2}Z_{3}Y_{4}
(- X_{2}X_{4}) (- X_{1}Y_{3}X_{4}Z_{5}) = X_{1}X_{2}Y_{3}Z_{5}
(+i Y_{2}Z_{3}X_{4}X_{5}Z_{6}) (-i X_{1}X_{2}X_{3}Y_{4}Y_{5}Z_{6}) = - X_{1}Z_{2}Y_{3}Z_{4}Z_{5}
(-i X_{1}Y_{2}Z_{5}X_{6}Y_{7}Z_{8}) (-i Y_{1}Y_{2}Z_{3}X_{4}X_{5}X_{7}) = -i Z_{1}Z_{3}X_{4}Y_{5}X_{6}Z_{7}Z_{8}
(- X_{2}X_{3}X_{4}Z_{5}Z_{6}Y_{7}Y_{8}Z_{10}) (- Y_{3}X_{4}X_{5}Y_{6}X_{7}Z_{8}) = +i X_{2}Z_{3}Y_{5}X_{6}Z_{7}X_{8}Z_{10}
(-i X_{1}X_{2}Z_{4}Z_{5}Z_{6}Y_{8}Z_{9}Z_{10}X_{11}X_{12}Z_{14}) (X_{2}X_{3}Y_{4}X_{5}Z_{6}Z_{7}Z_{8}X_{9}Z_{10}Z_{11}Z_{12}X_{13}X_{14}) = X_{1}X_{3}X_{4}Y_{5}Z_{7}X_{8}Y_{9}Y_{11}Y_{12}X_{13}Y_{14}
(- X_{3}X_{4}Z_{5}X_{6}Z_{7}Y_{8}Y_{10}Z_{12}Z_{14}X_{17}Z_{18}) (-i Z_{1}X_{2}X_{3}X_{4}Y_{5}X_{6}Z_{8}Z_{9}X_{10}Z_{11}Y_{13}Z_{15}Y_{16}

### I.2. Krylov Space Construction

In [178]:
def krylov_space(H, tol=1e-10):
    basis = [pc.pauli_identity(H.N)]
    def inner(A, B):
        return (A @ B).trace()/2**H.N
    while True:
        # Generate next potential basis vector H^n
        Knew = H @ basis[-1]
        # Gram-Schmidt orthogonalization against existing basis
        for K in basis:
            proj = inner(K, Knew).item()
            Knew = Knew - proj * K
        # Check if new vector is linearly independent
        norm = inner(Knew, Knew).real.item()
        if norm < tol:
            break
        # Add normalized vector to basis
        basis.append(Knew/numpy.sqrt(norm).item())
    dim = len(basis)
    for i, A in enumerate(basis):
        for j, B in enumerate(basis):
            for k, C in enumerate(basis):
                ope = inner(C, A @ B).real.round(decimals=15)
                if ope != 0.:
                    print("({},{},{}): {}".format(i, j, k, ope))
    return basis

In [179]:
H = - pc.pauli('XX') - pc.pauli('ZZ')
krylov_space(H)

(0,0,0): 1.0
(0,1,1): 1.0
(0,2,2): 1.0
(1,0,1): 1.0
(1,1,0): 1.0
(1,1,2): -1.0
(1,2,1): -1.0
(2,0,2): 1.0
(2,1,1): -1.0
(2,2,0): 1.0


[1 II, -0.71 ZZ -0.71 XX, -1 YY]

In [180]:
H = - pc.pauli('ZZII') - pc.pauli('IZZI') - pc.pauli('IIZZ') - pc.pauli('XXXX')
krylov_space(H)

(0,0,0): 1.0
(0,1,1): 1.0
(0,2,2): 1.0
(0,3,3): 1.0
(0,4,4): 1.0
(1,0,1): 1.0
(1,1,0): 1.0
(1,2,3): 0.408248290463863
(1,3,2): 0.408248290463863
(2,0,2): 1.0
(2,1,3): 0.408248290463863
(2,2,0): 1.0
(2,2,2): 1.632993161855453
(2,2,4): 1.0
(2,3,1): 0.408248290463863
(2,4,2): 1.0
(3,0,3): 1.0
(3,1,2): 0.408248290463863
(3,2,1): 0.408248290463863
(3,3,0): 1.0
(4,0,4): 1.0
(4,2,2): 1.0
(4,4,0): 1.0


[1 IIII,
 -0.50 IIZZ -0.50 IZZI -0.50 ZZII -0.50 XXXX,
 0.41 IZIZ +0.41 ZIZI +0.41 ZZZZ -0.41 XXYY -0.41 XYYX -0.41 YYXX,
 -0.50 ZIIZ +0.50 XYXY +0.50 YXYX -0.50 YYYY,
 -1 YXXY]

## II. Random Clifford Circuit

### II.1. Random Clifford Unitary Forward

In [266]:
def random_clifford_forward(N, nop, seed=42):
    numpy.random.seed(seed)
    gs = numpy.random.randint(0, 2, (nop, 2*N))
    ps = numpy.random.randint(0, 4, (nop,))
    ops = pc.PauliList(gs, ps)
    map = pc.random_clifford_map(N)
    head = sum([["X_{"+str(i+1)+"} &\mapsto", "Z_{"+str(i+1)+"} &\mapsto"] for i in range(N)],[])
    for h, t in zip(head, to_latex(map)):
        print(h+" "+t+" \\\\")
    return to_latex(ops), to_latex(ops.transform_by(map))

random_clifford_forward(10, 6)

X_{1} &\mapsto - X_{1}Z_{2}Y_{3}Y_{4}X_{5}Z_{6}Y_{7}Y_{8}Y_{9}Y_{10} \\
Z_{1} &\mapsto X_{1}Z_{3}Y_{4}Z_{5}Y_{6}Y_{7}Y_{8} \\
X_{2} &\mapsto - X_{1}X_{2}Y_{3}Z_{5}X_{7}X_{9} \\
Z_{2} &\mapsto - Y_{2}Z_{3}Y_{6}X_{7}X_{8}Z_{9}Y_{10} \\
X_{3} &\mapsto - Z_{1}Y_{2}Z_{4}Z_{5}Z_{6}Y_{7}X_{8}Z_{9}Y_{10} \\
Z_{3} &\mapsto Y_{1}X_{2}Z_{4}Z_{5}Z_{6}Y_{7}X_{8}Z_{10} \\
X_{4} &\mapsto X_{1}Y_{2}Z_{3}Z_{4}X_{5}Y_{9}X_{10} \\
Z_{4} &\mapsto X_{1}X_{3}Y_{6}Y_{7}Z_{8}X_{10} \\
X_{5} &\mapsto Y_{1}Y_{3}Y_{5}X_{6}X_{7}Z_{8}X_{9} \\
Z_{5} &\mapsto Z_{1}Z_{2}Z_{3}X_{4}Z_{5}X_{6}X_{7}Y_{8}Z_{9}Z_{10} \\
X_{6} &\mapsto X_{1}X_{2}Y_{5}Z_{8}Y_{9}X_{10} \\
Z_{6} &\mapsto X_{1}X_{2}Z_{4}Z_{5}X_{6}Y_{7}Z_{9}Z_{10} \\
X_{7} &\mapsto X_{1}Z_{3}Y_{6}Z_{7}Z_{8} \\
Z_{7} &\mapsto X_{1}Z_{2}X_{3}Y_{6}X_{7}Y_{8}X_{10} \\
X_{8} &\mapsto - X_{1}Y_{2}Y_{3}X_{4}Y_{5}Y_{6}X_{7}X_{9} \\
Z_{8} &\mapsto X_{1}X_{2}X_{7}X_{8}X_{9} \\
X_{9} &\mapsto - X_{2}X_{3}X_{5}Z_{6}Y_{7}X_{8}Y_{9}X_{10} \\
Z_{9} &\mapsto - Z_{3}Z_{4}X_{5}Z_

(['-i Z_{1}Z_{3}Z_{5}X_{8}Y_{9}X_{10}',
  'X_{1}Y_{2}Y_{3}Y_{4}Y_{5}Y_{7}X_{8}X_{9}',
  '+i Y_{2}Y_{3}X_{4}Y_{5}Z_{6}Z_{7}Z_{8}X_{9}',
  '- Z_{3}X_{4}Y_{5}Y_{6}Z_{7}Z_{8}Y_{9}Z_{10}',
  'Z_{1}Z_{2}X_{4}Y_{5}Y_{6}Y_{7}Y_{8}Y_{9}X_{10}',
  '-i Z_{1}Y_{2}Y_{3}Y_{4}X_{5}X_{6}Y_{7}Z_{8}Z_{9}X_{10}'],
 ['-i Z_{1}Z_{2}Y_{3}Y_{6}Y_{7}Y_{8}Y_{9}Z_{10}',
  '- X_{1}Y_{2}X_{3}X_{4}X_{5}Y_{6}X_{7}X_{8}Y_{10}',
  '+i X_{1}Z_{2}Z_{3}X_{4}X_{5}Y_{6}Y_{8}X_{9}Y_{10}',
  'Z_{1}Z_{2}Z_{5}Y_{7}X_{8}Y_{9}Z_{10}',
  '- Z_{1}Y_{3}Z_{4}X_{5}Z_{6}X_{7}Y_{8}Y_{9}',
  '-i X_{1}Z_{2}X_{3}X_{5}Y_{7}Y_{8}Z_{9}X_{10}'])

### II.2. Random Clifford Unitary Backward

In [309]:
def random_clifford_backward(N, nop, seed=32):
    numpy.random.seed(seed)
    gs = numpy.random.randint(0, 2, (nop, 2*N))
    ps = numpy.random.randint(0, 4, (nop,))
    ops = pc.PauliList(gs, ps)
    map = pc.random_clifford_map(N)
    head = sum([["X_{"+str(i+1)+"} &\mapsto", "Z_{"+str(i+1)+"} &\mapsto"] for i in range(N)],[])
    for h, t in zip(head, to_latex(map)):
        print(h+" "+t+" \\\\")
    return to_latex(ops), to_latex(ops.transform_by(map.inverse()))

random_clifford_backward(10, 6)

X_{1} &\mapsto - Z_{1}Z_{3}X_{6}Y_{7}X_{8}X_{9}Y_{10} \\
Z_{1} &\mapsto - Y_{1}Y_{2}Y_{3}Y_{5}X_{6}X_{7}Y_{10} \\
X_{2} &\mapsto Y_{1}Y_{2}Y_{3}Y_{4}Z_{5}Y_{6}X_{7}X_{9}Y_{10} \\
Z_{2} &\mapsto - Y_{1}X_{4}Y_{5}Z_{6}X_{7}Y_{8}Z_{9}X_{10} \\
X_{3} &\mapsto - X_{1}Z_{2}X_{4}X_{5}Z_{6}Z_{8}Y_{9} \\
Z_{3} &\mapsto - Y_{1}Y_{2}Y_{3}X_{5}X_{6}Z_{7}Y_{9} \\
X_{4} &\mapsto Z_{1}Y_{2}Y_{3}Y_{5}Y_{7}Z_{9} \\
Z_{4} &\mapsto Y_{1}Z_{2}Z_{3}X_{4}Z_{5}X_{8}X_{9}Z_{10} \\
X_{5} &\mapsto - Y_{1}Y_{2}X_{3}X_{4}Z_{5}Z_{7}X_{8}Z_{10} \\
Z_{5} &\mapsto X_{1}Y_{2}Y_{4}Y_{5}X_{7}Z_{9}X_{10} \\
X_{6} &\mapsto X_{1}Y_{5}Z_{6}Y_{7}X_{8}Y_{9}X_{10} \\
Z_{6} &\mapsto - Z_{1}X_{2}X_{3}Z_{4}Y_{5}X_{6}Z_{7}Y_{8}Y_{9}Y_{10} \\
X_{7} &\mapsto X_{1}Y_{2}Z_{3}Z_{5}Z_{6}X_{7}X_{8}Z_{9}Y_{10} \\
Z_{7} &\mapsto Z_{2}Y_{3}Y_{5}X_{6}Y_{7}Y_{8}X_{9}Y_{10} \\
X_{8} &\mapsto X_{1}X_{2}Y_{4}Y_{5}Z_{6}Y_{7}Z_{8}Z_{9} \\
Z_{8} &\mapsto X_{1}X_{2}Z_{3}Z_{5}Z_{7}Y_{9}X_{10} \\
X_{9} &\mapsto - Z_{2}Y_{3}X_{4}Z_{7}Y_{8}Z_{9}Y_{10} \

(['+i Y_{1}X_{2}Z_{4}Z_{5}Z_{6}Y_{7}Z_{8}Z_{9}Y_{10}',
  'X_{1}X_{2}Z_{6}Y_{7}Z_{8}X_{9}X_{10}',
  'Z_{1}Z_{2}Z_{3}Z_{4}Z_{7}X_{9}X_{10}',
  '+i Y_{1}Z_{2}Z_{3}Y_{4}Y_{5}Y_{6}Y_{8}X_{9}',
  'Y_{2}Z_{5}X_{7}X_{8}Y_{9}',
  'Z_{2}Y_{3}X_{4}Z_{5}X_{6}Y_{7}'],
 ['-i Y_{2}Y_{3}Y_{4}Z_{5}Z_{6}X_{7}Z_{8}X_{10}',
  '- X_{1}Z_{2}X_{3}Z_{4}Y_{5}Z_{7}Z_{8}Y_{9}X_{10}',
  '- X_{1}Z_{3}X_{4}Y_{5}Z_{6}Y_{7}Y_{8}X_{9}',
  '-i Y_{1}Y_{2}X_{3}X_{4}Y_{5}Z_{8}Y_{9}Y_{10}',
  'X_{1}Y_{2}Z_{3}Z_{4}Z_{5}Y_{7}Z_{8}X_{9}Z_{10}',
  '- Y_{1}X_{2}X_{3}X_{4}Z_{5}X_{6}X_{7}X_{8}Y_{9}X_{10}'])

In [291]:
from numba import njit

In [312]:
@njit
def f(seed):
    numpy.random.seed(seed)
    return numpy.random.randint(0, 2, 4)

f(42)

array([0, 1, 0, 0])

In [313]:
numpy.random.seed(42)
numpy.random.get_state()

('MT19937',
 array([        42, 3107752595, 1895908407, 3900362577, 3030691166,
        4081230161, 2732361568, 1361238961, 3961642104,  867618704,
        2837705690, 3281374275, 3928479052, 3691474744, 3088217429,
        1769265762, 3769508895, 2731227933, 2930436685,  486258750,
        1452990090, 3321835500, 3520974945, 2343938241,  928051207,
        2811458012, 3391994544, 3688461242, 1372039449, 3706424981,
        1717012300, 1728812672, 1688496645, 1203107765, 1648758310,
         440890502, 1396092674,  626042708, 3853121610,  669844980,
        2992565612,  310741647, 3820958101, 3474052697,  305511342,
        2053450195,  705225224, 3836704087, 3293527636, 1140926340,
        2738734251,  574359520, 1493564308,  269614846,  427919468,
        2903547603, 2957214125,  181522756, 4137743374, 2557886044,
        3399018834, 1348953650, 1575066973, 3837612427,  705360616,
        4138204617, 1604205300, 1605197804,  590851525, 2371419134,
        2530821810, 4183626679, 2872

In [320]:
numpy.random.seed(42)
numpy.random.randint(0, 1, 6)
numpy.random.get_state()

('MT19937',
 array([        42, 3107752595, 1895908407, 3900362577, 3030691166,
        4081230161, 2732361568, 1361238961, 3961642104,  867618704,
        2837705690, 3281374275, 3928479052, 3691474744, 3088217429,
        1769265762, 3769508895, 2731227933, 2930436685,  486258750,
        1452990090, 3321835500, 3520974945, 2343938241,  928051207,
        2811458012, 3391994544, 3688461242, 1372039449, 3706424981,
        1717012300, 1728812672, 1688496645, 1203107765, 1648758310,
         440890502, 1396092674,  626042708, 3853121610,  669844980,
        2992565612,  310741647, 3820958101, 3474052697,  305511342,
        2053450195,  705225224, 3836704087, 3293527636, 1140926340,
        2738734251,  574359520, 1493564308,  269614846,  427919468,
        2903547603, 2957214125,  181522756, 4137743374, 2557886044,
        3399018834, 1348953650, 1575066973, 3837612427,  705360616,
        4138204617, 1604205300, 1605197804,  590851525, 2371419134,
        2530821810, 4183626679, 2872